# Custom plotting library fully in Python!

In [ ]:
from math import pi

import numpy as np

import branca

from ipywidgets import VBox, IntSlider

from ipycanvas import Canvas, hold_canvas

In [ ]:
def init_2d_plot(x, y, color=None, scheme=branca.colormap.linear.RdBu_11, canvas=None, canvas_size=(800, 600), padding=0.1):
    if canvas is None:
        canvas = Canvas(size=canvas_size)
    else:
        canvas.size = canvas_size

    padding_x = padding * canvas_size[0]
    padding_y = padding * canvas_size[1]

    drawarea = (drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y) = (padding_x, padding_y, canvas_size[0] - 2 * padding_x, canvas_size[1] - 2 * padding_y)

    min_x, min_y, max_x, max_y = np.min(x), np.min(y), np.max(x), np.max(y)

    dx = max_x - min_x
    dy = max_y - min_y

    # Turns a data coordinate into pixel coordinate
    scale_x = lambda x: drawarea_max_x * (x - min_x) / dx + drawarea_min_x
    scale_y = lambda y: drawarea_max_y * (1 - (y - min_y) / dy) + drawarea_min_y

    # Turns a pixel coordinate into data coordinate
    unscale_x = lambda sx: (sx - drawarea_min_x) * dx / drawarea_max_x + min_x
    unscale_y = lambda sy: (1 - ((sy - drawarea_min_y) / drawarea_max_y)) * dy + min_y

    colormap = None
    if color is not None:
        colormap = scheme.scale(np.min(color), np.max(color))        

    return canvas, drawarea, scale_x, scale_y, unscale_x, unscale_y, colormap

In [ ]:
def draw_background(canvas, drawarea, unscale_x, unscale_y):
    drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y = drawarea

    # Draw background
    canvas.fill_style = '#f7f7f7'
    canvas.global_alpha = 0.3
    canvas.fill_rect(drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y)
    canvas.global_alpha = 1

    # Draw grid and ticks
    n_lines = 10
    canvas.fill_style = 'black'
    canvas.stroke_style = '#8c8c8c'
    canvas.line_width = 1
    canvas.begin_path()

    for i in range(n_lines):
        j = i / (n_lines - 1)
        line_x = drawarea_max_x * j + drawarea_min_x
        line_y = drawarea_max_y * j + drawarea_min_y

        # Line on the y axis
        canvas.move_to(line_x, drawarea_min_y)
        canvas.line_to(line_x, drawarea_max_y + drawarea_min_y)

        # Line on the x axis
        canvas.move_to(drawarea_min_x, line_y)
        canvas.line_to(drawarea_max_x + drawarea_min_x, line_y)

        # Draw y tick
        canvas.text_align = 'right'
        canvas.text_baseline = 'middle'
        canvas.fill_text('{0:.2e}'.format(unscale_y(line_y)), drawarea_min_x * 0.95, line_y)

        # Draw x tick
        canvas.text_align = 'center'
        canvas.text_baseline = 'top'
        canvas.fill_text('{0:.2e}'.format(unscale_x(line_x)), line_x, drawarea_max_y + drawarea_min_y + drawarea_min_y * 0.05)

    canvas.stroke()
    canvas.close_path()

In [ ]:
def scatter_plot(x, y, size, color, scheme=branca.colormap.linear.RdBu_11, stroke_color='black', canvas=None):
    canvas, drawarea, scale_x, scale_y, unscale_x, unscale_y, colormap = init_2d_plot(x, y, color, scheme, canvas=canvas)

    with hold_canvas(canvas):
        canvas.clear()
        canvas.save()

        draw_background(canvas, drawarea, unscale_x, unscale_y)

        # Draw scatter
        n_marks = min(x.shape[0], y.shape[0], size.shape[0], color.shape[0])

        canvas.stroke_style = stroke_color

        for idx in range(n_marks):
            canvas.begin_path()

            canvas.fill_style = colormap(color[idx])

            canvas.arc(
                scale_x(x[idx]), scale_y(y[idx]),
                size[idx],
                0, 2 * pi
            )

            canvas.fill()
            canvas.stroke()

            canvas.close_path()

        canvas.restore()

    return canvas

In [ ]:
def line_plot(x, y, line_color='#749cb8', line_width=2, canvas=None):
    canvas, drawarea, scale_x, scale_y, unscale_x, unscale_y, _ = init_2d_plot(x, y, canvas=canvas)

    with hold_canvas(canvas):
        canvas.clear()
        canvas.save()

        draw_background(canvas, drawarea, unscale_x, unscale_y)

        # Draw lines
        n_points = min(x.shape[0], y.shape[0])

        canvas.begin_path()
        canvas.stroke_style = line_color
        canvas.line_width = line_width
        canvas.line_join = 'bevel'
        canvas.line_cap = 'round'
        canvas.move_to(scale_x(x[0]), scale_y(y[0]))
        for idx in range(1, n_points):
            canvas.line_to(
                scale_x(x[idx]), scale_y(y[idx])
            )

        canvas.stroke()
        canvas.close_path()
        
        canvas.restore()

    return canvas

In [ ]:
def heatmap_plot(x, y, color, scheme=branca.colormap.linear.RdBu_11, canvas=None):
    canvas, drawarea, scale_x, scale_y, unscale_x, unscale_y, colormap = init_2d_plot(x, y, color, scheme, canvas=canvas)

    outof_x_bound = lambda idx: True if idx >= x.shape[0] or idx < 0 else False
    outof_y_bound = lambda idx: True if idx >= y.shape[0] or idx < 0 else False

    with hold_canvas(canvas):
        canvas.clear()
        canvas.save()

        draw_background(canvas, drawarea, unscale_x, unscale_y)

        # Draw heatmap
        n_marks = min(x.shape[0], y.shape[0])

        for x_idx in range(1, color.shape[0] - 1):
            for y_idx in range(1, color.shape[1] - 1):
                canvas.fill_style = colormap(color[x_idx][y_idx])

                rect_center = (scale_x(x[x_idx]), scale_y(y[y_idx]))
                neighbours_x = (scale_x(x[x_idx - 1]), scale_x(x[x_idx + 1]))
                neighbours_y = (scale_y(y[y_idx - 1]), scale_y(y[y_idx + 1]))

                rect_top_left_corner = ((neighbours_x[0] + rect_center[0]) / 2, (neighbours_y[0] + rect_center[1]) / 2)
                rect_low_right_corner = ((neighbours_x[1] + rect_center[0]) / 2, (neighbours_y[1] + rect_center[1]) / 2)

                width = rect_low_right_corner[0] - rect_top_left_corner[0]
                height = rect_low_right_corner[1] - rect_top_left_corner[1]

                canvas.fill_rect(
                    rect_top_left_corner[0], rect_top_left_corner[1],
                    width, height
                )

        canvas.restore()

    return canvas

# Scatter plot

In [ ]:
n_points = 500

In [ ]:
x = np.random.rand(n_points)
y = np.random.rand(n_points)
sizes = np.random.randint(2, 8, n_points)
colors = np.random.rand(n_points) * 10 - 2

plot = scatter_plot(x, y, sizes, colors, branca.colormap.linear.viridis, stroke_color='white')
plot

### Because it's a Canvas, you can draw on top of it! 

In [ ]:
plot.stroke_style = 'red'
plot.line_width = 2
plot.stroke_rect(200, 300, 50, 100)

# Line plot

In [ ]:
x = np.linspace(0, 20, 500)
y = np.sin(x)

line_plot(x, y, line_width=3)

In [ ]:
slider = IntSlider(min=1, max=10, step=1)

x = np.linspace(-20, 20, 500)
y = np.power(x, slider.value)

power_plot = line_plot(x, y, line_color='#32a852', line_width=3)

def on_slider_change(change):
    y = np.power(x, slider.value)

    line_plot(x, y, line_color='#32a852', line_width=3, canvas=power_plot)

slider.observe(on_slider_change, 'value')

VBox((power_plot, slider))

In [ ]:
n = 1_000
x = np.linspace(0, 100, n)
y = np.cumsum(np.random.randn(n))

line_plot(x, y, line_width=3)

# Heatmap

In [ ]:
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
x_grid, y_grid = np.meshgrid(x, y)
color = np.sin(x_grid + y_grid**2) + np.cos(x_grid**2 + y_grid**2)

heatmap_plot(x, y, color, scheme=branca.colormap.linear.RdYlBu_05)